In [2]:
import os
import re
import glob

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [27]:
PARLAMINT_DIR="/Volumes/lhe-drive-1/datasets/ParlaMint-5.0"
REDUCED_PARLAMINT_DIR = "/Volumes/lhe-drive-1/datasets/ParlaMint-5.0-Reduced"

In [ ]:
all_speeches_df = pd.read_csv(f"{REDUCED_PARLAMINT_DIR}/all_speeches.csv")
all_speeches_df

/var/folders/2p/pyn18v1x4wj1wqxyg2035jcr0000gn/T/ipykernel_21807/495172511.py:1: DtypeWarning: Columns (5,6,7,8,22) have mixed types. Specify dtype option on import or set low_memory=False.
  all_speeches_df = pd.read_csv("all_speeches.csv") #, low_memory=False)


,Text_ID,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic,country,year,speech_path
0,ParlaMint-HR-en_2017-12-12-0,ParlaMint-HR_2017-12-12-0.u56065,Minutes of the National Assembly of the Republ...,2017-12-12,Unicameralism,9. mandat,-,Zasedanje 6,2017-12-15,-,...,Opposition,Centre-right to right,PetrovBožo,"Petrov, Božo",M,1979,Other,HR,2017,/ParlaMint-HR-en.ana/ParlaMint-HR-en.txt/2017/...
1,ParlaMint-HR-en_2017-12-12-0,ParlaMint-HR_2017-12-12-0.u56066,Minutes of the National Assembly of the Republ...,2017-12-12,Unicameralism,9. mandat,-,Zasedanje 6,2017-12-15,-,...,Coalition,Centre-right,JandrokovićGordan,"Jandroković, Gordan",M,1967,Other,HR,2017,/ParlaMint-HR-en.ana/ParlaMint-HR-en.txt/2017/...
2,ParlaMint-HR-en_2017-12-12-0,ParlaMint-HR_2017-12-12-0.u56067,Minutes of the National Assembly of the Republ...,2017-12-12,Unicameralism,9. mandat,-,Zasedanje 6,2017-12-15,-,...,Coalition,-,BunjacBranimir,"Bunjac, Branimir",M,1972,Foreign Trade,HR,2017,/ParlaMint-HR-en.ana/ParlaMint-HR-en.txt/2017/...
3,ParlaMint-HR-en_2017-12-12-0,ParlaMint-HR_2017-12-12-0.u56068,Minutes of the National Assembly of the Republ...,2017-12-12,Unicameralism,9. mandat,-,Zasedanje 6,2017-12-15,-,...,Coalition,Centre-right,JandrokovićGordan,"Jandroković, Gordan",M,1967,Other,HR,2017,/ParlaMint-HR-en.ana/ParlaMint-HR-en.txt/2017/...
4,ParlaMint-HR-en_2017-12-12-0,ParlaMint-HR_2017-12-12-0.u56070,Minutes of the National Assembly of the Republ...,2017-12-12,Unicameralism,9. mandat,-,Zasedanje 6,2017-12-15,-,...,Opposition,Centre-right to right,PetrovBožo,"Petrov, Božo",M,1979,International Affairs,HR,2017,/ParlaMint-HR-en.ana/ParlaMint-HR-en.txt/2017/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744264,ParlaMint-NL-en_2022-03-23-tweedekamer-7,ParlaMint-NL_2022-03-23-tweedekamer-7.u111,Report of the meeting of the Dutch Lower House...,2022-03-23,Lower house,Meeting of the 30th Tweede Kamer,Session 7,Meeting 63,Sitting 2022-03-23,-,...,Coalition,Centre,VeraBergkamp,"Bergkamp, Vera",F,1971,Other,NL,2022,/ParlaMint-NL-en.ana/ParlaMint-NL-en.txt/2022/...
4744265,ParlaMint-NL-en_2022-03-23-tweedekamer-7,ParlaMint-NL_2022-03-23-tweedekamer-7.u112,Report of the meeting of the Dutch Lower House...,2022-03-23,Lower house,Meeting of the 30th Tweede Kamer,Session 7,Meeting 63,Sitting 2022-03-23,-,...,Coalition,Centre,VeraBergkamp,"Bergkamp, Vera",F,1971,Other,NL,2022,/ParlaMint-NL-en.ana/ParlaMint-NL-en.txt/2022/...
4744266,ParlaMint-NL-en_2022-03-23-tweedekamer-7,ParlaMint-NL_2022-03-23-tweedekamer-7.u113,Report of the meeting of the Dutch Lower House...,2022-03-23,Lower house,Meeting of the 30th Tweede Kamer,Session 7,Meeting 63,Sitting 2022-03-23,-,...,Opposition,Centre-left to left,JesseKlaver,"Klaver, Jesse",M,1986,Other,NL,2022,/ParlaMint-NL-en.ana/ParlaMint-NL-en.txt/2022/...
4744267,ParlaMint-NL-en_2022-03-23-tweedekamer-7,ParlaMint-NL_2022-03-23-tweedekamer-7.u114,Report of the meeting of the Dutch Lower House...,2022-03-23,Lower house,Meeting of the 30th Tweede Kamer,Session 7,Meeting 63,Sitting 2022-03-23,-,...,Opposition,Right,PieterOmtzigt,"Omtzigt, Pieter",M,1974,Social Welfare,NL,2022,/ParlaMint-NL-en.ana/ParlaMint-NL-en.txt/2022/...


In [11]:
sampled_speeches_df = all_speeches_df.sample(1000)

In [23]:
environment_speeches = sampled_speeches_df[sampled_speeches_df['Topic'] == "Environment"]
environment_speeches.iloc[0]

Text_ID                                      ParlaMint-NO-en_2020-12-14
ID                                ParlaMint-NO_2020-12-14.ana.ud115e418
Title                 Hansard from meeting in the Norwegian Parliame...
Date                                                         2020-12-14
Body                                                      Unicameralism
Term                                                     Term 2017-2021
Session                                               Session 2020-2021
Meeting                                                               -
Sitting                                          Sitting day 2020-12-14
Agenda                                                                -
Subcorpus                                                         COVID
Lang                                                            English
Speaker_role                                                    Regular
Speaker_MP                                                      

In [37]:
_text_cache = {}

def get_speech(session_path: str, speech_id: str) -> str:
  global _text_cache
  filepath = f"{REDUCED_PARLAMINT_DIR}{session_path}"
  if filepath not in _text_cache:
    _text_cache[filepath] = {}
    with open(filepath, 'r', encoding='utf-8') as f:
      for line in f:
        if line.strip():
          parts = line.strip().split(None, 1)
          if len(parts) == 2:
            _text_cache[filepath][parts[0]] = parts[1]
  
  return _text_cache[filepath].get(speech_id)

get_speech(environment_speeches.iloc[0]["speech_path"], environment_speeches.iloc[0]["ID"])


'Marine littering is one of the major problems we see, perhaps one of the biggest, and the government has arranged to combat plastics in the sea. When the Progress Party was in government, we had our own aid program for marine pollution, and we prioritised helping countries that had problems with their waste management. This entry has now been greatly reduced. How can the Left believe that the control of plastics in the sea is a high priority when the sum for this purpose is shaved so strongly?'

In [ ]:
def get_corpus_for_speeches(speeches: pd.DataFrame):
  corpus = ""
  for _, row in speeches.iterrows():
    corpus += get_speech(row["speech_path"], row["ID"]) + "\n"
  return corpus

get_corpus_for_speeches(environment_speeches)

'Marine littering is one of the major problems we see, perhaps one of the biggest, and the government has arranged to combat plastics in the sea. When the Progress Party was in government, we had our own aid program for marine pollution, and we prioritised helping countries that had problems with their waste management. This entry has now been greatly reduced. How can the Left believe that the control of plastics in the sea is a high priority when the sum for this purpose is shaved so strongly?\nWhen we consult the subsidy scheme, we will also come again with a letter. We will make clear what, as far as we are concerned, are the categories covered by the exceptions. And out of my mind: for the environmental zones, I think we already had the carnival in between. But I shall look at it further at the request of Mr Graus. Mr. Chairman. Then the third block, sustainability and circular economy. That is a subject that Members of Parliament know about for a long time, since 2010 when I becam